# Dealing with class imbalance

Class imbalance is a major problem in any __"real-world"__ classification task.

Assume you are dealing with classifying violence vs non-violence scenarios from video sequences streaming from a security camera, and the task of this camera is to identify __violence__ vs __non-Violence__ scenarios. We all know that 
$99.9\%$ of the time, security camera record the "normal" (non-violence) scenarios while detecting "violence" behavior can be considered as a rare event.

Similarly in the medical diagnosis use case for a rare disease, in this case we generally dealing with highly "imbalance" samples.

In this tutorial you will learn about:
    
    1. Effect of class imbalance on your model training
    2. Two major techniques to deal with the class imbalance:
        a. Oversampling
        b. Penalization
        c. Oversampling + Penalization
    3. Metrics to effectly monitor your training procedure.

In this tutorial similar to the provious one we will work with the Malaria Dataset. However, to simulate the class imbalance we extract a subset from the dataset such that the ratio between the positive class "infected" and negative class "uninfected" is 100/1000 for training and 10/90 for the validation set.

# Steps to establish a machine learning pipeline for this tutorial
1. Dataloading and shaping an imbalance dataset
2. Torch ready data prepration
3. Model definition
4. Optimizer and loss definition
5. Training and validation

In [33]:
import torch
import os
import pandas as pd
import numpy as np


from monai.data import DataLoader, Dataset
from torch.utils.data.sampler import WeightedRandomSampler
target = df_train['label'].to_numpy()
from monai.transforms import (
    Compose,
    LoadImaged, 
    AddChanneld, 
    RandFlipd, 
    RandRotate90d, 
    Resized, 
    EnsureChannelFirstd,
    RandGaussianNoised,
    ToTensord
    )
import torch.optim as optim
import torch.nn as nn

## Dataloading and creating an imbalance dataset

In [54]:
root_dir = '/Users/sadeghmh/Desktop/Projects/data/cell_images/'
infected_name = 'Parasitized'
uninfected_name = 'Uninfected'
df_infected = pd.DataFrame({'image_name':os.listdir(os.path.join(root_dir,infected_name)), 
                          'folder_name': [infected_name for _ in range(len(os.listdir(os.path.join(root_dir,infected_name))))],
                         'label': [1 for _ in range(len(os.listdir(os.path.join(root_dir,infected_name))))]})

df_uninfected = pd.DataFrame({'image_name':os.listdir(os.path.join(root_dir,uninfected_name)), 
                            'folder_name': [uninfected_name for _ in range(len(os.listdir(os.path.join(root_dir,uninfected_name))))],
                             'label': [0 for _ in range(len(os.listdir(os.path.join(root_dir,uninfected_name))))]})

# select the subset of the data
df_infected_sample = df_infected.sample(n=100, random_state=42).reset_index()
df_uninfected_sample = df_uninfected.sample(n= 1000, random_state=42).reset_index()

# Create the train/test split
# train
df_infected_train = df_infected_sample.loc[0:89]
df_uninfected_train = df_uninfected_sample.loc[0:909]
df_list = [df_infected_train, df_uninfected_train]
df_train = pd.concat(df_list).reset_index()
# test
df_infected_test = df_infected_sample.loc[90:]
df_uninfected_test = df_uninfected_sample.loc[910:]

df_list = [df_infected_test, df_uninfected_test]
df_val = pd.concat(df_list).reset_index()


print('number of infected samples is {} and number of unifected is {}'.format(len(df_infected), len(df_uninfected)))
print('number of train samples is {} and number of test is {}'.format(len(df_train), len(df_val)))



number of infected samples is 13780 and number of unifected is 13780
number of train samples is 1000 and number of test is 100


In [55]:
num_infected = len(df_train[df_train['label'] == 1])
num_uninfected = len(df_train[df_train['label'] == 0])
df_num = pd.DataFrame({in})
print(num_infected, num_uninfected)

90 910


In [35]:
target = df_train['label'].to_numpy()
class_sample_count = np.array(
    [len(np.where(target == t)[0]) for t in np.unique(target)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in target])
samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

In [30]:
target = df_train['label'].to_numpy()

## Helper function to shape the training and validation set in an acceptable Monai format

In [6]:
def list2dict(df):
    data_dict = {}
    list_of_dict = []
    for i in range(0,len(df)):
        #data_dict['image'] = os.path.join(root_dir,df.iloc[i,1],df.iloc[i,0])
        data_dict['image'] = os.path.join(root_dir,df['folder_name'][i],df['image_name'][i])
        data_dict['label'] = df['label'][i]
        list_of_dict.append(data_dict.copy())
    return list_of_dict
data_train = list2dict(df_train)
data_val = list2dict(df_val)

### Torch ready dataprepration

In [36]:


train_transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        Resized(keys=["image"],spatial_size=(224,224)),
        RandFlipd(keys=["image"], prob=0.5),
        RandRotate90d(keys=["image"], prob=0.5),
        RandGaussianNoised(keys=["image"], prob=0.2),
        ToTensord(keys=["image"])
])

val_transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        Resized(keys=["image"],spatial_size=(224,224)),
        ToTensord(keys=["image"])
])

ds_tr = Dataset(data=data_train,transform= train_transforms) # create dataset
ds_tr_loader = DataLoader(ds_tr, batch_size=100, sampler =sampler)

ds_val = Dataset(data=data_val,transform= val_transforms) # create dataset
ds_val_loader = DataLoader(ds_val, batch_size=100)

len(ds_tr), len(ds_val) 

(1000, 100)

In [38]:
for i, batch in enumerate(ds_tr_loader):
    target = batch['label']
    print ("batch index {}, 0/1: {}/{}".format(
        i,
        len(np.where(target.numpy() == 0)[0]),
        len(np.where(target.numpy() == 1)[0])))

batch index 0, 0/1: 45/55
batch index 1, 0/1: 50/50
batch index 2, 0/1: 45/55
batch index 3, 0/1: 52/48
batch index 4, 0/1: 43/57
batch index 5, 0/1: 47/53
batch index 6, 0/1: 60/40
batch index 7, 0/1: 44/56
batch index 8, 0/1: 42/58
batch index 9, 0/1: 59/41


# Define model, we use Resnet 50 for this purpose!

In [39]:
net = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
net.fc = nn.Linear(in_features=2048, out_features=2) # only last year needed to be replace

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
print()

Using cache found in /Users/sadeghmh/.cache/torch/hub/pytorch_vision_v0.9.0


## Define Optimizer and loss function

In [40]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

# training/validation

In [44]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
net.train()

tracker = dict()
accuracy_tr = list()
loss_tr = list()
accuracy_val = list()
loss_val = list()
auc_tr = list()
auc_val = list()

epochs = 10
for epoch in range(epochs):
    pred_iter = list()
    target_iter = list()
    accuracy_tr_iter = list()
    running_loss_tr = 0
    total_data_tr = 0
    running_loss_te = 0
    total_data_te = 0
    for i, batch in enumerate(ds_tr_loader):
        data = batch['image'].to(device)
        target = batch['label'].to(device)
        outputs = net(data)
        _,preds = torch.max(outputs,1)
        optimizer.zero_grad()
        loss = criterion(outputs, target)

        loss.backward()
        optimizer.step()
        
        pred_iter.extend(preds.cpu().numpy())
        target_iter.extend(target.cpu().numpy())
        running_loss_tr += loss.item() * data.size(0)
        total_data_tr += data.size(0)
    
    epoch_loss = running_loss_tr / total_data_tr
    loss_tr.append(epoch_loss)
    
    fpr, tpr, thresholds = roc_curve(target_iter, pred_iter)
    AUC = auc(fpr, tpr)
    ACC = accuracy_score(target_iter,pred_iter)
    auc_tr.append(AUC)
    accuracy_tr.append(ACC)
    
    print('training Phase: epoch: {} Loss: {:.4f} Acc: {:.4f} AUC: {:.4f}'.format(
                epoch , epoch_loss, ACC, AUC))
    net.eval()
    for i, batch in enumerate(ds_val_loader):
        data = batch['image'].to(device)
        target = batch['label'].to(device)
        outputs = net(data)
        _,preds = torch.max(outputs,1)
        loss = criterion(outputs, target)
        
        running_loss_te += loss.item() * data.size(0)
        total_data_te += data.size(0)
    epoch_loss = running_loss_te / total_data_te
    loss_val.append(epoch_loss)
    
    fpr, tpr, thresholds = roc_curve(target.cpu().numpy(), preds.cpu().numpy())
    AUC = auc(fpr, tpr)
    Acc = accuracy_score(target,preds)
    auc_val.append(AUC)
    accuracy_val.append(Acc)
    print('validation Phase: epoch: {} Loss: {:.2f} Acc: {:.2f} AUC: {:.2f}'.format(
                epoch , epoch_loss, ACC, AUC))

tracker['loss_tr'] = loss_tr
tracker['loss_val'] = loss_val
tracker['accuracy_tr'] = accuracy_tr
tracker['accuracy_val'] = accuracy_val
tracker['auc_tr'] = auc_tr
tracker['auc_val'] = auc_val
        
        

run
run
run
run
run
run
run
run
run
run
training Phase: epoch: 0 Loss: 0.1621 Acc: 0.9510 AUC: 0.9508


TypeError: unsupported format string passed to list.__format__